# 📘 UMAP Sampling Method Inference
This notebook generates synthetic samples using common sampling strategies (Grid, Random, LHS, Sobol),
reduces their dimensionality using **UMAP**, and infers the likely sampling method used based on structure.

It also exports:
- A 2D interactive HTML plot
- A static PNG image

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial import KDTree
from scipy.stats import entropy
import umap
import plotly.express as px
import plotly.io as pio

/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generate synthetic sampling data
np.random.seed(42)
n = 100
grid = np.array(np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))).T.reshape(-1, 2)
lhs = qmc.LatinHypercube(d=2).random(n)
sobol = qmc.Sobol(d=2).random(n)
rand = np.random.rand(n, 2)

X = np.vstack([grid, rand, lhs, sobol])
labels = ['Grid'] * len(grid) + ['Random'] * n + ['LHS'] * n + ['Sobol'] * n
df = pd.DataFrame(X, columns=['x1', 'x2'])
df['TrueSampling'] = labels

/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/scipy/stats/_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


In [3]:
# UMAP + KMeans
X_scaled = StandardScaler().fit_transform(df[['x1', 'x2']])
X_umap = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42).fit_transform(X_scaled)
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_umap)
df_umap = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])
df_umap['Cluster'] = clusters
df_umap['TrueSampling'] = df['TrueSampling']

/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [4]:
# Inference function
def classify_sampling(embedding, clusters):
    dists = KDTree(embedding).query(embedding, k=2)[0][:, 1]
    std = np.std(dists)
    sil = silhouette_score(embedding, clusters)
    ent = entropy(np.histogram(dists, bins=30, density=True)[0])
    if std < 0.05 and sil > 0.6:
        return 'Grid', 'Low std and high silhouette -> Grid'
    if std > 0.2 and sil < 0.3:
        return 'Random', 'High std and low silhouette -> Random'
    if 0.05 <= std <= 0.15:
        return ('Sobol', 'Moderate spread, low entropy -> Sobol') if ent < 2.0 else ('LHS', 'Moderate spread, higher entropy -> LHS')
    return 'Uncertain', 'Pattern unclear'

In [5]:
# Classify and print result
method, explanation = classify_sampling(X_umap, clusters)
print(f'Inferred Sampling Method: {method}')
print(f'Justification: {explanation}')

Inferred Sampling Method: LHS
Justification: Moderate spread, higher entropy -> LHS


In [6]:
!pip install -U kaleido

In [7]:
# Plot and export
fig = px.scatter(df_umap, x='UMAP1', y='UMAP2', color='Cluster', symbol='TrueSampling',
                 title=f'UMAP 2D Projection - Inferred: {method}', hover_data=['TrueSampling'])
fig.write_html('umap_2d_sampling_plot.html')
# pio.write_image(fig, 'umap_2d_sampling_plot.png', width=1000, height=800)
fig.show()

/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.


